<a href="https://colab.research.google.com/github/AmbikaRahulSingh/Spam_filter_for_Quora_questions/blob/main/Project_3_Spam_filter_for_Quora_questions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Importing required libraries

import warnings
warnings.filterwarnings('ignore')

import os
import time
import pandas as pd
import numpy as np
import math

import tensorflow as tf
from tensorflow import keras
from tqdm import tqdm

from sklearn.model_selection import train_test_split
from sklearn import metrics

import nltk
nltk.download('punkt')
from nltk import word_tokenize
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from keras.preprocessing.sequence import pad_sequences

from tensorflow.keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding, Conv1D, SpatialDropout1D, Bidirectional,GlobalMaxPool1D
from keras import initializers, regularizers, constraints, optimizers, layers
from tensorflow.keras.models import Model

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Glove 840B 300d embeddings

In [2]:
!wget http://nlp.stanford.edu/data/glove.840B.300d.zip
!unzip glove.840B.300d.zip
!rm glove.840B.300d.zip

--2021-05-24 15:29:23--  http://nlp.stanford.edu/data/glove.840B.300d.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.840B.300d.zip [following]
--2021-05-24 15:29:24--  https://nlp.stanford.edu/data/glove.840B.300d.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.840B.300d.zip [following]
--2021-05-24 15:29:24--  http://downloads.cs.stanford.edu/nlp/data/glove.840B.300d.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2176768927 (2.0G) [application/zip

In [4]:
!wget https://www.dropbox.com/sh/kpf9z73woodfssv/AAAwZ5DDt-aHwqZFHYrZ-ZBHa/train.csv

--2021-05-24 15:37:57--  https://www.dropbox.com/sh/kpf9z73woodfssv/AAAwZ5DDt-aHwqZFHYrZ-ZBHa/train.csv
Resolving www.dropbox.com (www.dropbox.com)... 162.125.5.18, 2620:100:601d:18::a27d:512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.5.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /sh/raw/kpf9z73woodfssv/AAAwZ5DDt-aHwqZFHYrZ-ZBHa/train.csv [following]
--2021-05-24 15:37:58--  https://www.dropbox.com/sh/raw/kpf9z73woodfssv/AAAwZ5DDt-aHwqZFHYrZ-ZBHa/train.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc38a7f6508308b38561b8f7284f.dl.dropboxusercontent.com/cd/0/inline/BPGusKcJku9PrixkMZkI3VI4CkF7f3ToHSO2syX66aG2bUaUzKxeYhASUgXebFvL6-gjSMTg1aS8Tj5jNV5FSxSTi8bW4YOm623TS560CnzEgEIPvt0VjJSoWKAs70SxiDUgChfRVAtpokU9ZFD1lLLc/file# [following]
--2021-05-24 15:37:58--  https://uc38a7f6508308b38561b8f7284f.dl.dropboxusercontent.com/cd/0/inline/BPGusKcJku9PrixkMZ

In [5]:
data=pd.read_csv('/content/train.csv')

In [6]:
data.shape

(1306122, 3)

In [7]:
data.isnull().sum()

qid              0
question_text    0
target           0
dtype: int64

In [8]:
data.head()

,qid,question_text,target
0,00002165364db923c7e6,How did Quebec nationalists see their province...,0
1,000032939017120e6e44,"Do you have an adopted dog, how would you enco...",0
2,0000412ca6e4628ce2cf,Why does velocity affect time? Does velocity a...,0
3,000042bf85aa498cd78e,How did Otto von Guericke used the Magdeburg h...,0
4,0000455dfa3e01eae3af,Can I convert montra helicon D to a mountain b...,0


In [9]:
data['target'].value_counts()

0    1225312
1      80810
Name: target, dtype: int64

In [10]:
train_df, val_df = train_test_split(data, test_size=0.2, random_state=2)

In [11]:
# !rm train.csv

In [12]:
embed_size = 300
max_features = 50000
maxlen = 100

train_X = train_df["question_text"].values
val_X = val_df["question_text"].values

train_y = train_df["target"].values
val_y = val_df["target"].values

In [13]:
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(train_X))
train_X = tokenizer.texts_to_sequences(train_X)
val_X = tokenizer.texts_to_sequences(val_X)

train_X = pad_sequences(train_X, maxlen=maxlen)
val_X = pad_sequences(val_X, maxlen=maxlen)

In [14]:
print(train_X.shape, val_X.shape)
print(train_y.shape, val_y.shape)

(1044897, 100) (261225, 100)
(1044897,) (261225,)


In [15]:
EMBEDDING_FILE = "/content/glove.840B.300d.txt"

In [16]:
def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')

In [17]:
embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE))

In [18]:
all_embs = np.stack(embeddings_index.values())
emb_mean,emb_std = all_embs.mean(), all_embs.std()
embed_size = all_embs.shape[1]

In [19]:
word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))
embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
for word, i in word_index.items():
    if i >= max_features: continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector

In [ ]:
# !rm glove.840B.300d.txt

In [20]:
inputs = Input(shape=(maxlen,))
layer = Embedding(max_features, embed_size,mask_zero=True, weights=[embedding_matrix],trainable=False)(inputs)
layer = Bidirectional(LSTM(64, return_sequences=True))(layer)
layer = GlobalMaxPool1D()(layer)
layer = Dense(16, activation="relu")(layer)
layer = Dropout(0.1)(layer)
layer = Dense(1, activation="sigmoid")(layer)
model = Model(inputs=inputs, outputs=layer)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 100)]             0         
_________________________________________________________________
embedding (Embedding)        (None, 100, 300)          15000000  
_________________________________________________________________
bidirectional (Bidirectional (None, 100, 128)          186880    
_________________________________________________________________
global_max_pooling1d (Global (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 16)                2064      
_________________________________________________________________
dropout (Dropout)            (None, 16)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17    

In [24]:
model.fit(train_X, train_y, batch_size=1000, epochs=1, validation_data=(val_X, val_y))

1045/1045 [==============================] - 3044s 3s/step - loss: 0.1174 - accuracy: 0.9528 - val_loss: 0.1066 - val_accuracy: 0.9577


In [25]:
pred = model.predict([val_X], batch_size=1000, verbose=1)

262/262 [==============================] - 306s 1s/step


In [26]:
pred.shape

(261225, 1)

In [27]:
cutoffs=np.linspace(0.001,0.999,999)

from sklearn.metrics import fbeta_score

fbetas=[]

for cutoff in cutoffs:
    
    predicted=(pred>cutoff).astype(int)
    
    fbetas.append(fbeta_score(val_y,predicted,2))
    
my_cutoff=cutoffs[fbetas==max(fbetas)]

# pred_test_y = model.predict([test_X], batch_size=1024, verbose=1)

# predictions = (pred_test_y>my_cutoff).astype(int)

# pd.Series(predictions).value_counts()

# submission = pd.DataFrame({"qid":val_df["qid"].values})

# submission['prediction'] = predictions

# submission.to_csv(('Ambika_Tupakula_P3.csv', index=False)

# note: preprocessing of test_X...needed before predicting

In [28]:
my_cutoff

array([0.136])

In [29]:
pred = (pred>my_cutoff).astype(int)

In [33]:
np.unique(pred,return_counts=True)

(array([0, 1]), array([233123,  28102]))

In [34]:
pred=pred.astype(int)

In [35]:
from sklearn.metrics import roc_auc_score

In [36]:
score = roc_auc_score(val_y,pred)

In [37]:
score

0.8868453077213354

In [38]:
# Saving the model 

from tensorflow.keras.models import model_from_json

model_json = model.to_json()

with open("model.json", "w") as json_file:

    json_file.write(model_json)

model.save_weights('model_weights.h5',overwrite = True)


In [39]:
# loading the model 

from tensorflow.keras.models import model_from_json

json_file = open('model.json', 'r')

loaded_model_json = json_file.read()
json_file.close()

loaded_model = model_from_json(loaded_model_json)

loaded_model.summary()

loaded_model.load_weights('model_weights.h5')


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 100)]             0         
_________________________________________________________________
embedding (Embedding)        (None, 100, 300)          15000000  
_________________________________________________________________
bidirectional (Bidirectional (None, 100, 128)          186880    
_________________________________________________________________
global_max_pooling1d (Global (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 16)                2064      
_________________________________________________________________
dropout (Dropout)            (None, 16)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17    